<a href="https://colab.research.google.com/github/AlkaidCheng/example/blob/master/CatBoostHParamTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install  --upgrade bayesian-optimization
! pip install ray[tune]

Requirement already up-to-date: bayesian-optimization in /usr/local/lib/python3.6/dist-packages (1.1.0)


In [0]:
!wget https://gitlab.cern.ch/clcheng/mlhep-googlesummerofcode/raw/master/Prerequisite/MachineLearning/QIS_EXAM_200Events.npz

--2020-03-08 07:59:52--  https://gitlab.cern.ch/clcheng/mlhep-googlesummerofcode/raw/master/Prerequisite/MachineLearning/QIS_EXAM_200Events.npz
Resolving gitlab.cern.ch (gitlab.cern.ch)... 188.185.68.13, 188.184.31.109, 188.184.104.112, ...
Connecting to gitlab.cern.ch (gitlab.cern.ch)|188.185.68.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9140 (8.9K) [application/zip]
Saving to: ‘QIS_EXAM_200Events.npz.1’

QIS_EXAM_200Events. 100%[===================>]   8.93K  --.-KB/s    in 0s      

2020-03-08 07:59:53 (170 MB/s) - ‘QIS_EXAM_200Events.npz.1’ saved [9140/9140]



In [0]:
import numpy as np
def shuffle_zippedarrays(arrays):
  shape = arrays[0].shape
  assert all(shape == arrays[0].shape for arr in arrays)
  index = np.random.permutation(shape[0])
  return [arr[index] for arr in arrays]

def shuffle_data(input, label):
  output = shuffle_zippedarrays([input,label])
  return output[0], output[1]

def load_data(arrays, shuffle = True):
  data = {}
  for key in arrays:
    input, label = np.array([]), np.array([])
    for klabel in arrays[key].item():
      _input = arrays[key].item()[klabel]
      _label = np.full((_input.shape[0],),int(klabel))
      input = np.concatenate((input,_input),axis=0) if input.size else _input
      label = np.concatenate((label,_label),axis=0) if label.size else _label
    if shuffle:
      input, label = shuffle_data(input, label)
    data[key] = {'input': input, 'label': label}
  return data
def load_train_test_input_labels(arrays, shuffle = True):
  data = load_data(arrays, shuffle)
  return data['training_input']['input'], data['test_input']['input'], data['training_input']['label'], data['test_input']['label']

In [0]:
data = np.load('QIS_EXAM_200Events.npz',allow_pickle=True)
train_input, test_input, train_label, test_label = load_train_test_input_labels(data)
X = np.concatenate((train_input,test_input),axis = 0)
Y = np.concatenate((train_label,test_label),axis = 0)

In [0]:
! pip install catboost

In [0]:
import catboost as cb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import ray
from hyperopt import hp
from ray import tune
from pdb import set_trace

def train_catboost(params):
    dataset = cb.Pool(X, Y)
    scores = cb.cv(
            pool=dataset,
            params=params,
            fold_count=2,
            verbose=False)
        
    # scores returns a dictionary with mean and std (per-fold) of metric 
    # value for each cv iteration, we choose minimal value of objective 
    # mean (though it will be better to choose minimal value among all folds)
    # because noise is additive
    loss = np.min(scores['test-AUC-mean'])
    tune.track.log(
        mean_accuracy=loss,
        done=True)
    
ray.shutdown()
ray.init(webui_host='127.0.0.1')

space = {
    'iterations': 100
    'depth': hyperopt.hp.randint('depth', 7),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 0.2, 1.0),
  #  'bagging_temperature': hp.uniform('bagging_temperature', 0.0, 1.0),
 #   'border_count': hp.quniform('border_count', 1, 255, 1),
    'l2_leaf_reg': hyperopt.hp.uniform('l2_leaf_reg', 1, 10),
    'loss_function': 'Logloss',
    'eval_metric': 'AUC', 
    'task_type' : 'GPU'
}


algo = HyperOptSearch(
        space,
        max_concurrent=4,
        metric="mean_accuracy",
        mode="min")

from ray.tune.schedulers import ASHAScheduler
! rm -r /catboost_result
analysis = tune.run(
    train_catboost,
    num_samples=10,
    verbose =  0,
    resources_per_trial={'gpu': 1},
    search_alg = algo,
    local_dir = '/catboost_result',
    scheduler=AsyncHyperBandScheduler(metric="mean_accuracy", mode="min"))
print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))

2020-03-08 08:04:57,836	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-08 08:04:57,843	INFO resource_spec.py:212 -- Starting Ray with 6.59 GiB memory available for workers and up to 3.3 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-08 08:04:58,143	INFO services.py:1078 -- View the Ray dashboard at 127.0.0.1:8265
2020-03-08 08:04:59,643	INFO function_runner.py:250 -- tune.track signature detected.
2020-03-08 08:11:48,614	WARNING worker.py:1058 -- The actor or task with ID ffffffffffffffff1b26bd520100 is pending and cannot currently be scheduled. It requires {CPU: 1.000000}, {GPU: 1.000000} for execution and {CPU: 1.000000}, {GPU: 1.000000} for placement, but this node only has remaining {GPU: 1.000000}, {node:172.28.0.2: 1.000000}, {CPU: 2.000000}, {memory: 6.591797 GiB}, {object_store_memory: 2.246094 GiB}. In total there are 0 pending ta

Best config:  {'bagging_temperature': 0.46868882132631184, 'border_count': 109.0, 'depth': 10.0, 'eval_metric': 'Logloss', 'iterations': 800.0, 'l2_leaf_reg': 7.0, 'learning_rate': 1.8616950937495608e-06, 'loss_function': 'Logloss', 'task_type': 'GPU'}
